#### Import Dependencies

In [8]:
import time
from langchain_ollama import ChatOllama
from typing import TypedDict, Optional
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END

####  Define State for the Interview

In [9]:
# Define the state structure for interview process
class GraphState(TypedDict):
    all_history: Optional[str] = None
    history: Optional[str] = None
    result: Optional[str] = None
    final_result: Optional[str] = None
    total_questions: Optional[int] = None
    interviewer: Optional[str] = None
    candidate: Optional[str] = None
    current_question: Optional[str] = None
    current_answer: Optional[str] = None
    round: Optional[int] = None
    total_rounds: Optional[int] = None
    total_questions_per_round: Optional[int] = None

#### Initialize Model

In [10]:
# Initialize Ollama Model
model = ChatOllama(
    model='llama3.1',   # Specify the model (Llama 3.1)
    temperature=0.8,    # Control creativity (higher is more creative)
    num_predict=256     # Limit the number of tokens generated
)

# Function to generate response from LLM
def llm(prompt: str) -> str:
    response = model.predict(prompt)  
    return response

#### Define Interview Prompts

In [11]:
# Interview prompts
prompt_interviewer = "You're an interviewer. You need to interview a human for the {} role. This is the interview so far:\n{}\nAsk your next question. Don't repeat questions. Output just the question."
prompt_result = "Evaluate the complete performance of the candidate. Provide a rating at the end of the interview from 1 to 10 and a one-line explanation. The interview:\n{}"
prompt_verdict = "Based on the entire interview, should the candidate be selected? Respond with 'Yes' or 'No' and provide a one-line reason. The interview:\n{}"

#### Define Interview Functions

In [12]:
# Function to handle AI's question generation
def handle_question(state):
    """ Generates a question for the candidate based on the role and conversation history """
    history = state.get('history', '').strip()
    role = state.get('interviewer', '').strip()
    candidate = state.get('candidate', '').strip()

    prompt = prompt_interviewer.format(role, candidate, history)
    question = llm(prompt)  

    print(f"\nAI Interviewer: {question}\n")
    time.sleep(0.5)  # Small pause for readability

    return {
        "history": history + '\n\n' + question,
        "current_question": question,
        "total_questions": state.get('total_questions', 0) + 1
    }

# Function to handle candidate's response
def handle_response(state):
    """ Collects the candidate's answer from user input """
    history = state.get('history', '').strip()
    question = state.get('current_question', '').strip()

    print(f"Question: {question}")
    answer = input("Your Answer: ")  

    return {
        "history": history + '\n\n' + answer,
        "current_answer": answer
    }

# Function to evaluate the interview and give feedback
def handle_result(state):
    """ Evaluates the candidate's overall performance and generates feedback """
    history = state.get('history', '').strip()
    interviewer = state.get('interviewer', '').strip()

    prompt = prompt_result.format(history)
    result = llm(prompt)
    all_history = state.get('all_history', '').strip()

    return {
        "result": state.get('result', '') + '\n' + result,
        "history": 'Nothing',
        "all_history": all_history + '\n\nInterviewed by ' + interviewer + '\n' + history,
        'final_result': result
    }

# Function to determine final selection decision
def handle_selection(state):
    """ Determines if the candidate is selected or not based on the interview performance """
    result = state.get('result', '').strip()
    prompt = prompt_verdict.format(result)
    verdict = llm(prompt)

    return {"final_result": verdict}


#### Define Workflow & Conditions

In [13]:
# Initialize workflow
workflow = StateGraph(GraphState)

# Add nodes to workflow
workflow.add_node("handle_question", handle_question)
workflow.add_node("handle_response", handle_response)
workflow.add_node("handle_result", handle_result)
workflow.add_node("handle_selection", handle_selection)

# Function to check if more questions are needed
def check_conv_length(state):
    """ Checks if the candidate has answered fewer than 5 questions """
    if state.get("total_questions", 0) < 5:
        return "handle_question"
    else:
        return "handle_result"

# Function to check if interview is complete
def check_rounds(state):
    """ Checks if the interview should continue or move to final selection """
    if state.get("total_questions", 0) >= 5 and state.get("total_rounds", 1) == 1:
        return "handle_selection"
    else:
        return "handle_question"

# Add conditional edges for handling responses
workflow.add_conditional_edges("handle_response", check_conv_length, {
    "handle_question": "handle_question",
    "handle_result": "handle_result"
})

# Add conditional edges for handling results
workflow.add_conditional_edges("handle_result", check_rounds, {
    "handle_question": "handle_question",
    "handle_selection": "handle_selection"
})

# Set the entry point for the workflow
workflow.set_entry_point("handle_question")
workflow.add_edge('handle_question', "handle_response")
workflow.add_edge('handle_selection', END)

# Compile the workflow
app = workflow.compile()

#### Run the Interview (run everytime)

In [14]:
# Get job description input
job_description = input("Enter the job description: ")

# Initialize interview state
conversation_state = {
    'total_questions': 0,
    'candidate': job_description,
    'total_rounds': 1,
}

# Run the interview
conversation_state = app.invoke(conversation_state)

# Print final feedback
final_feedback = conversation_state.get("final_result")
print(f"\nFinal Feedback: {final_feedback}")
print(conversation_state['result'])
print(conversation_state['all_history'])


AI Interviewer: What do you think are some important qualities that make a good classmate?

Question: What do you think are some important qualities that make a good classmate?

AI Interviewer: What do you think are some qualities that make a good friend?

Question: What do you think are some qualities that make a good friend?

AI Interviewer: What do you think are some important qualities that a good friend should have?

Question: What do you think are some important qualities that a good friend should have?

AI Interviewer: What do you think are some qualities that make a good friend?

Question: What do you think are some qualities that make a good friend?

AI Interviewer: Can you describe a time when you had to work with others on a group project?

Question: Can you describe a time when you had to work with others on a group project?

Final Feedback: No. The candidate's lack of depth in thinking, inability to provide real-world examples, and repetition throughout the interview outw